In [7]:
# Import libraries.
using Turing, StatsPlots, Random

┌ Info: Precompiling StatsPlots [f3b207a7-027a-5e70-b257-86293d7955fd]
└ @ Base loading.jl:1278


In [101]:
# Declare our Turing model.
@model function agent(other_agent, action, other_agents_depth=0, depth=0, discrete_sampler = PG, discrete_sampler_hyper_param=10, num_of_iterations=10)
    if depth > 0 
        opp_action_chain = sample(other_agent(agent, action, depth-1, other_agents_depth), discrete_sampler(discrete_sampler_hyper_param), num_of_iterations, progress=false);
        opp_alpha_1 = mean(opp_action_chain[:"alpha[1]"])
        opp_alpha_2 = mean(opp_action_chain[:"alpha[2]"])
        opp_alpha_3 = 1 - opp_alpha_1 - opp_alpha_2
        alpha ~ Dirichlet([opp_alpha_3, opp_alpha_1 , opp_alpha_2])
    else
        # Our prior belief about the probability of RPS.
        alpha ~ Dirichlet(ones(3)/3)
    end
    action ~ Categorical(vec(alpha))
    return action
end

agent (generic function with 6 methods)

In [102]:
chain = sample(agent(agent, 1.0, 1, 1, PG, 10, 10), PG(1), 10 , progress = false);
chain

Chains MCMC chain (10×5×1 Array{Float64,3}):

Log evidence      = -4.6426763145628485
Iterations        = 1:10
Thinning interval = 1
Chains            = 1
Samples per chain = 10
parameters        = alpha[1], alpha[2], alpha[3]
internals         = logevidence, lp

Summary Statistics
  parameters      mean       std   naive_se      mcse       ess      rhat 
      Symbol   Float64   Float64    Float64   Missing   Float64   Float64 

    alpha[1]    0.2390    0.3257     0.1030   missing    8.1888    1.0059
    alpha[2]    0.4522    0.3783     0.1196   missing    7.4255    1.0009
    alpha[3]    0.3089    0.3396     0.1074   missing   11.3177    0.8962

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

    alpha[1]    0.0000    0.0019    0.0517    0.4138    0.8612
    alpha[2]    0.0106    0.1562    0.3739    0.8190    0.9799
    alpha[3]    0.0001    0.0389    0.1253    0.6155    0.8337
